### LangChain表达式

In [1]:
# Sample code

from langchain.prompts import ChatPromptTemplate # 导入模板类
from langchain.chat_models import ChatOpenAI 
from langchain.schema.output_parser import StrOutputParser # 导入字符串解析器

# 实例化提示词模板和聊天模型包装器
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
model = ChatOpenAI(
    openai_api_base="http://localhost:8000/v1",
    openai_api_key="none",
    streaming=True,
    verbose=True,
    model_name="Qwen/Qwen1.5-4B-Chat"
)

# 定义处理链
chain = prompt | model | StrOutputParser() # 使用管道操作符 | 连接模块
# 调用处理链
response = chain.invoke({"topic":"bears"}) # 请注意，invoke方法传入的参数是一个字典，其中每一个键值对应提示词内的占位符

response

/home/elin/anaconda3/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


'Why did the bear cross the playground? To get to the slide!'

### 提示词模板+模型包装器

In [2]:
# 重新定义处理链
chain = prompt | model 

# 调用处理链
response = chain.invoke({"topic":"bears"})

response

AIMessage(content='Why did the bear put on his party hat?\n\nBecause he wanted to get everyone "polarized."')

In [3]:
# 使用带有stop的新链
chain = prompt | model.bind(stop=["\n"]) # 请注意，stop仅仅代表模型的输出将抵达的终点
response = chain.invoke({"topic":"bears"})
response


AIMessage(content='Why did the bear refuse to move? ')

In [4]:
# 由于bind方法同样支持OpenAI的函数回调功能，可以将函数描述列表绑定到模型包装器上
functions = [
    {
        "name":"joke",
        "description":"A joke",
        "parameters":{
            "type":"object",
            "properties":{
                "setup":{
                    "type":"string",
                    "description":"The setup for the joke"
                }
            },
            "punchline":{
                "type":"string",
                "description":"The punchline for the joke"
            },
            "required":["setup","punchline"]
        }
    }
]

# 重新定义处理链
chain = prompt | model.bind(function_call = {"name":"joke"},functions=functions)

response = chain.invoke({"topic":"bears"},config={})

response

AIMessage(content="Why couldn't the bear sit on the rug? Because it was flat as a pancake!")

### 提示词模板+模型包装器+输出解析器

In [5]:
# 为链导入字符串解析器
chain = prompt | model | StrOutputParser()
response = chain.invoke({"topic":"bears"},config={})
response

'Why did the bear refuse to move? \nBecause it was stuck in the frozen bear river.'

### 多功能组合链

In [7]:
from operator import itemgetter
# 第一个提示词用于获取一个人来自于哪个城市
prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
# 第二个提示词用于获取一个城市所在的国家
prompt2 = ChatPromptTemplate.from_template("What country is the city {city} in?respond in {language}")
# 组合第一个链，用于获取城市
chain1 = prompt1 | model | StrOutputParser() # 限制使用StrOutputParser()来控制输出仅为字符串
# 组合第二个链，其中输入分别是第一个链返回的城市信息和从invoke中拿到的语言信息
# 接着将输入的参数填充进第二个提示词中
# 最后模型输出，并交由字符串解析器处理
chain2 = {"city":chain1,"language":itemgetter("language")} | prompt2 | model | StrOutputParser()

chain2.invoke({"person":"obama","language":"chinese"})

'美国总统巴拉克·奥巴马来自美国夏威夷州火奴鲁鲁市。'